<a href="https://colab.research.google.com/github/Pathway2008/CarPrice/blob/main/base_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. "배터리-", 주행+, "보증-"
Niro, eT, MS, MY, ID4, Soul, MX, KNE, Tay

2. '배터리+', '주행-', '보증+'
RSeTGT, i5, ION6, i3, ION5

3. 배터리, 거리, 보증, 연식의 영향이 미비함
IX, IONIQ, M3

4. 나머지는 개별실행
TayGTS, Q4eT, TayCT, EV6
'''

In [ ]:
'''
오차가 심한 차종 - IONIQ , Tay, TayCT
배터리 용량이 is nan값들이 not nan값에 비해 오차가 큼
'''

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib.font_manager as fm
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [2]:
!apt-get -qq install fonts-nanum


fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 123635 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [89]:
train = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/train.csv')
test = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/CarPrice/new_car/sample_submission.csv')

In [90]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   object 
 2   모델        7497 non-null   object 
 3   차량상태      7497 non-null   object 
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   object 
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   object 
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(3), object(6)
memory usage: 644.4+ KB


In [91]:
ordinal_features = ['제조사', '차량상태', '구동방식','사고이력',  '모델']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train[feature])
    train[feature] = le.transform(train[feature])

    for label in np.unique(test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[feature] = le.transform(test[feature])

In [97]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7497 entries, 0 to 7496
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        7497 non-null   object 
 1   제조사       7497 non-null   int64  
 2   모델        7497 non-null   int64  
 3   차량상태      7497 non-null   int64  
 4   배터리용량     4786 non-null   float64
 5   구동방식      7497 non-null   int64  
 6   주행거리(km)  7497 non-null   int64  
 7   보증기간(년)   7497 non-null   int64  
 8   사고이력      7497 non-null   int64  
 9   연식(년)     7497 non-null   int64  
 10  가격(백만원)   7497 non-null   float64
dtypes: float64(2), int64(8), object(1)
memory usage: 644.4+ KB


In [224]:
model_1 = ['Niro', 'eT', 'MS', 'MY', 'ID4', 'Soul', 'MX', 'KNE', 'Tay']
model_2 = ['RSeTGT', 'i5', 'ION6', 'i3', 'ION5']
model_3 = ['iX', 'IONIQ', 'M3']
model_4 = ['TayGTS', 'Q4eT', 'TayCT', 'EV6']

In [225]:
# 모델 이름과 숫자 매핑 확인
model_mapping = dict(enumerate(le.classes_))

# 숫자 값으로 변환된 모델 리스트 생성
model_1_labels = [key for key, value in model_mapping.items() if value in model_1]
model_2_labels = [key for key, value in model_mapping.items() if value in model_2]
model_3_labels = [key for key, value in model_mapping.items() if value in model_3]
model_4_labels = [key for key, value in model_mapping.items() if value in model_4]

In [216]:
model_mapping

{0: 'EV6',
 1: 'ID4',
 2: 'ION5',
 3: 'ION6',
 4: 'IONIQ',
 5: 'KNE',
 6: 'M3',
 7: 'MS',
 8: 'MX',
 9: 'MY',
 10: 'Niro',
 11: 'Q4eT',
 12: 'RSeTGT',
 13: 'Soul',
 14: 'Tay',
 15: 'TayCT',
 16: 'TayGTS',
 17: 'eT',
 18: 'i3',
 19: 'i5',
 20: 'iX'}

In [226]:
train_1 = train[train['모델'].isin(model_1_labels)].reset_index(drop=True)
train_2 = train[train['모델'].isin(model_2_labels)].reset_index(drop=True)
train_3 = train[train['모델'].isin(model_3_labels)].reset_index(drop=True)
train_4 = train[train['모델'].isin(model_4_labels)].reset_index(drop=True)

In [227]:
len(train_1), len(train_2), len(train_3), len(train_4)

(3335, 1919, 786, 1457)

In [53]:
# def split_by_model(data):
#     model_counts = data['모델'].value_counts()
#     train_splits = []
#     test_splits = []

#     for model_label in model_counts.index:
#         model_data = data[data['모델'] == model_label]

#         # 비율 유지하여 split
#         model_train, model_test = train_test_split(
#             model_data,
#             test_size=0.2,  # 원하는 비율 (80:20)
#             random_state=42
#         )

#         train_splits.append(model_train)
#         test_splits.append(model_test)

#     # 최종 데이터프레임 결합
#     train_final = pd.concat(train_splits).reset_index(drop=True)
#     test_final = pd.concat(test_splits).reset_index(drop=True)
#     return train_final, test_final

In [54]:
# from sklearn.model_selection import train_test_split
# train_1_train, train_1_test = split_by_model(train_1)
# train_2_train, train_2_test = split_by_model(train_2)
# train_3_train, train_3_test = split_by_model(train_3)

In [42]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00


In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt

In [228]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [229]:
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
cat_model = CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=42, verbose=0)
lgb_model = LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42)

models = {
    'XGBoost': xgb_model,
    'CatBoost': cat_model,
    'LightGBM': lgb_model
}

# Voting Regressors
voting_models = {
    'XGB+Cat': VotingRegressor([('xgb', xgb_model), ('cat', cat_model)]),
    'XGB+LGB': VotingRegressor([('xgb', xgb_model), ('lgb', lgb_model)]),
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)]),
    'XGB+Cat+LGB': VotingRegressor([('xgb', xgb_model), ('cat', cat_model), ('lgb', lgb_model)])
}

In [230]:
all_models = {**models, **voting_models}

In [231]:
def evaluate_with_stratified_kfold(X, y, model_column, models):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model_rmse_scores = {name: [] for name in models.keys()}
    error_analysis = {name: [] for name in models.keys()}  # 오차 저장
    all_predictions = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, model_column)):
        print(f"Fold {fold + 1}")

        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        for model_name, model in models.items():
            print(f"Training {model_name}...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            score = rmse(y_val, y_pred)
            model_rmse_scores[model_name].append(score)
            print(f"{model_name} RMSE: {score:.4f}")

            # 오차 분석 저장
            error_analysis[model_name].append(y_val - y_pred)
            all_predictions.append({
                'model': model_name,
                'true_values': y_val,
                'predictions': y_pred,
                'errors': y_val - y_pred
            })

    return model_rmse_scores, error_analysis, all_predictions

In [232]:
datasets = {
    'train_1': (train_1.drop(columns=['ID', '가격(백만원)']), train_1['가격(백만원)'], train_1['모델']),
    'train_2': (train_2.drop(columns=['ID', '가격(백만원)']), train_2['가격(백만원)'], train_2['모델']),
    'train_3': (train_3.drop(columns=['ID', '가격(백만원)']), train_3['가격(백만원)'], train_3['모델']),
    'train_4': (train_4.drop(columns=['ID', '가격(백만원)']), train_4['가격(백만원)'], train_4['모델'])
}
final_results = {}
error_results = {}
all_predictions = []

In [233]:
for dataset_name, (X, y, model_column) in datasets.items():
    print(f"\nEvaluating on {dataset_name} dataset...")
    scores, errors, predictions = evaluate_with_stratified_kfold(X, y, model_column, all_models)
    final_results[dataset_name] = scores
    error_results[dataset_name] = errors
    all_predictions.extend(predictions)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [234]:
for dataset_name, scores in final_results.items():
    print(f"\n{dataset_name} Results:")
    for model_name, model_scores in scores.items():
        print(f"{model_name}: {np.mean(model_scores):.4f} ± {np.std(model_scores):.4f}")



train_1 Results:
XGBoost: 1.2668 ± 0.0486
CatBoost: 1.1403 ± 0.0743
LightGBM: 1.1898 ± 0.0831
XGB+Cat: 1.1821 ± 0.0592
XGB+LGB: 1.2050 ± 0.0586
Cat+LGB: 1.1460 ± 0.0739
XGB+Cat+LGB: 1.1705 ± 0.0624

train_2 Results:
XGBoost: 0.4479 ± 0.0156
CatBoost: 0.4197 ± 0.0119
LightGBM: 0.4344 ± 0.0139
XGB+Cat: 0.4239 ± 0.0145
XGB+LGB: 0.4352 ± 0.0144
Cat+LGB: 0.4195 ± 0.0134
XGB+Cat+LGB: 0.4236 ± 0.0142

train_3 Results:
XGBoost: 2.7341 ± 0.1663
CatBoost: 2.6012 ± 0.1872
LightGBM: 2.6805 ± 0.0929
XGB+Cat: 2.6157 ± 0.1643
XGB+LGB: 2.6589 ± 0.1243
Cat+LGB: 2.6093 ± 0.1280
XGB+Cat+LGB: 2.6130 ± 0.1362

train_4 Results:
XGBoost: 1.9521 ± 0.1380
CatBoost: 1.8424 ± 0.0931
LightGBM: 1.8090 ± 0.0965
XGB+Cat: 1.8315 ± 0.0977
XGB+LGB: 1.8302 ± 0.0963
Cat+LGB: 1.7874 ± 0.0958
XGB+Cat+LGB: 1.7987 ± 0.0915


In [113]:
X_train_1 = train_1.drop(columns=['ID', '가격(백만원)'])
y_train_1 = train_1['가격(백만원)']
model_column_1 = train_1['모델']

# Stratified Split (모델 컬럼의 비율 유지)
X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
    X_train_1, y_train_1, test_size=0.2, random_state=42, stratify=model_column_1)

In [131]:
xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
cat_model = CatBoostRegressor(n_estimators=500, learning_rate=0.05, random_state=42, verbose=0)
lgb_model = LGBMRegressor(n_estimators=500, learning_rate=0.05, random_state=42)

models = {
    'XGBoost': xgb_model,
    'CatBoost': cat_model,
    'LightGBM': lgb_model
}

# Voting Regressors
voting_models = {
    'Cat+LGB': VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])}

In [118]:
# 예측값과 RMSE 값을 저장할 리스트
predictions = {}
rmse_scores = {}

# 각 모델에 대해 예측 및 RMSE 계산
for model_name, model in all_models.items():
    # 모델 학습
    model.fit(X_train_stratified, y_train_stratified)

    # 예측
    y_pred = model.predict(X_valid_stratified)

    # RMSE 계산
    rmse_value = np.sqrt(mean_squared_error(y_valid_stratified, y_pred))

    # 예측값과 RMSE 값 저장
    predictions[model_name] = y_pred
    rmse_scores[model_name] = rmse_value

    print(f"{model_name} RMSE: {rmse_value:.4f}")

XGBoost RMSE: 1.2560
CatBoost RMSE: 1.2009
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000303 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 347
[LightGBM] [Info] Number of data points in the train set: 2668, number of used features: 9
[LightGBM] [Info] Start training from score 54.370139
LightGBM RMSE: 1.2919
XGB+Cat RMSE: 1.2016
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000243 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 347
[LightGBM] [Info] Number of data points in the train set: 2668, number of used features: 9
[LightGBM] [Info] Start training from score 54.370139
XGB+LGB RMSE: 1.2433
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was

In [128]:
y_pred

array([ 82.61951128,  40.01418082,  28.18648074,  22.80394052,
       114.59961496,  84.62215691,  23.24249674,  21.70697274,
        74.33355047,  41.19262171,  22.7948532 ,  69.56513819,
        27.78933092,  99.56797507, 114.84515332,  23.2482996 ,
        34.02765505,  22.71242687,  24.69605773,  65.31259205,
        98.76558041,  24.92052681,  28.11885137,  70.03020713,
        72.40923077,  24.79752576,  88.14124802,  27.78703562,
        23.97790366,  27.93484266,  28.13800382,  78.0933312 ,
        77.92451167,  68.29962843,  24.81331236,  65.75147941,
        36.89524611,  27.69585672,  20.4149415 ,  24.71577646,
        90.61814521,  27.62081414,  78.79752866,  28.09394117,
        41.03679488,  64.25699479,  81.34959316, 100.23943533,
        59.9422272 ,  33.81160507,  22.97652335,  40.24840619,
        40.03729575,  27.90809695,  78.00915561,  66.69383807,
        71.70269805,  23.04766414,  59.80259544,  28.21273647,
        21.72593357,  82.96284412,  33.91715002,  63.75

In [134]:
cat_model.fit(X_train_stratified, y_train_stratified)

voting_model = VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
voting_model.fit(X_train_stratified, y_train_stratified)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 347
[LightGBM] [Info] Number of data points in the train set: 2668, number of used features: 9
[LightGBM] [Info] Start training from score 54.370139


VotingRegressor(estimators=[('cat',
                             <catboost.core.CatBoostRegressor object at 0x7ce5a4c9d5a0>),
                            ('lgb',
                             LGBMRegressor(learning_rate=0.05, n_estimators=500,
                                           random_state=42))])

In [136]:
y_pred_cat = cat_model.predict(X_valid_stratified)
y_pred_voting = voting_model.predict(X_valid_stratified)

In [140]:
result_df = pd.concat([X_valid_stratified, y_valid_stratified], axis=1)
result_df['CatBoost_Pred'] = y_pred_cat
result_df['Voting_Pred'] = y_pred_voting
result_df['Cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['voting-real'] = ( (result_df['가격(백만원)'] - result_df['Voting_Pred'])**2)**0.5

In [141]:
result_df

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),CatBoost_Pred,Voting_Pred,Cat-real,voting-real
1026,5,7,1,NaN,0,46360,3,1,0,82.62,82.133650,82.542708,0.486350,0.077292
732,6,1,2,57.861,0,138245,1,0,0,40.38,40.066516,40.073193,0.313484,0.306807
258,2,5,1,68.488,1,48994,7,0,1,27.12,28.129530,28.174038,1.009530,1.054038
2714,3,13,2,46.000,0,53354,1,0,0,23.07,22.897403,22.802672,0.172597,0.267328
1598,4,14,2,54.849,0,199819,2,0,0,114.41,114.731744,114.672696,0.321744,0.262696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442,3,13,1,56.000,0,18277,5,0,0,23.37,23.250791,22.950707,0.119209,0.419293
1863,5,8,1,NaN,0,47509,5,0,1,93.86,93.727266,93.775813,0.132734,0.084187
1229,6,1,1,65.400,0,34913,4,0,0,40.75,41.183585,41.126096,0.433585,0.376096
2063,3,10,0,NaN,1,418,10,0,0,22.06,21.667445,21.800183,0.392555,0.259817


In [142]:
# 모델 컬럼을 기준으로 그룹화하여 평균과 표준편차 계산
model_stats = result_df.groupby('모델')[['Cat-real', 'voting-real']].agg(['mean', 'std'])
model_stats

Cat-real           voting-real          
        mean       std        mean       std
모델                                          
1   0.523631  0.306842    0.531704  0.324656
5   0.506625  0.318161    0.516525  0.326466
7   0.556085  0.381242    0.576631  0.361765
8   0.494552  0.349448    0.503509  0.323833
9   0.690847  0.470015    0.859530  0.827931
10  0.506653  0.312402    0.488260  0.292830
13  0.535149  0.306423    0.577672  0.301368
14  2.549924  1.927517    2.429964  2.026077
17  0.379283  0.281762    0.403925  0.326823

In [155]:
X_train_3 = train_3.drop(columns=['ID', '가격(백만원)'])
y_train_3 = train_3['가격(백만원)']
model_column_3 = train_3['모델']

# Stratified Split (모델 컬럼의 비율 유지)
X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
    X_train_3, y_train_3, test_size=0.2, random_state=42, stratify=model_column_3)

In [156]:
cat_model.fit(X_train_stratified, y_train_stratified)

voting_model = VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
voting_model.fit(X_train_stratified, y_train_stratified)

y_pred_cat = cat_model.predict(X_valid_stratified)
y_pred_voting = voting_model.predict(X_valid_stratified)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000116 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 273
[LightGBM] [Info] Number of data points in the train set: 628, number of used features: 9
[LightGBM] [Info] Start training from score 58.917707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [157]:
result_df = pd.concat([X_valid_stratified, y_valid_stratified], axis=1)
result_df['CatBoost_Pred'] = y_pred_cat
result_df['Voting_Pred'] = y_pred_voting
result_df['Cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['voting-real'] = ( (result_df['가격(백만원)'] - result_df['Voting_Pred'])**2)**0.5

In [158]:
model_stats = result_df.groupby('모델')[['Cat-real', 'voting-real']].agg(['mean', 'std'])
model_stats

Cat-real           voting-real          
        mean       std        mean       std
모델                                          
4   4.373104  3.100409    4.660487  3.153759
6   0.361009  0.271815    0.405857  0.287000
20  0.510664  0.323576    0.505383  0.328714

In [235]:
X_train_4 = train_4.drop(columns=['ID', '가격(백만원)'])
y_train_4 = train_4['가격(백만원)']
model_column_4 = train_4['모델']

# Stratified Split (모델 컬럼의 비율 유지)
X_train_stratified, X_valid_stratified, y_train_stratified, y_valid_stratified = train_test_split(
    X_train_4, y_train_4, test_size=0.2, random_state=42, stratify=model_column_4)

In [236]:
cat_model.fit(X_train_stratified, y_train_stratified)

voting_model = VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
voting_model.fit(X_train_stratified, y_train_stratified)

y_pred_cat = cat_model.predict(X_valid_stratified)
y_pred_voting = voting_model.predict(X_valid_stratified)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 303
[LightGBM] [Info] Number of data points in the train set: 1165, number of used features: 9
[LightGBM] [Info] Start training from score 96.091983


In [237]:
result_df = pd.concat([X_valid_stratified, y_valid_stratified], axis=1)
result_df['CatBoost_Pred'] = y_pred_cat
result_df['Voting_Pred'] = y_pred_voting
result_df['Cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['voting-real'] = ( (result_df['가격(백만원)'] - result_df['Voting_Pred'])**2)**0.5

In [238]:
model_stats = result_df.groupby('모델')[['Cat-real', 'voting-real']].agg(['mean', 'std'])
model_stats

Cat-real           voting-real          
        mean       std        mean       std
모델                                          
0   0.589199  0.373576    0.529668  0.355878
11  0.380995  0.309799    0.416142  0.300230
15  2.936283  2.482113    2.758579  2.310288
16  0.424441  0.560119    0.378333  0.377338

In [242]:
len(result_df[(result_df['모델']==15)]) , len(result_df[(result_df['모델']==15) & (result_df['배터리용량'].isna())]), len(result_df[(result_df['모델']==15) & (~result_df['배터리용량'].isna())])

(67, 25, 42)

In [241]:
result_df[(result_df['모델']==15) & (result_df['배터리용량'].isna())]['Cat-real'].mean(),result_df[(result_df['모델']==15) & (~result_df['배터리용량'].isna())]['Cat-real'].mean()

(5.36498267820846, 1.4906289166027171)

In [245]:
result_df[(result_df['모델']==16) & (result_df['배터리용량'].isna())]['Cat-real'].mean(),result_df[(result_df['모델']==16) & (~result_df['배터리용량'].isna())]['Cat-real'].mean()

(0.519234141608771, 0.3875768115609201)

In [162]:
train_ioniq = train[train['모델']==4].drop('ID',axis=1).reset_index(drop=True)

In [196]:
X_train_ioniq = train_ioniq.drop(columns=['가격(백만원)'])
y_train_ioniq = train_ioniq['가격(백만원)']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_ioniq, y_train_ioniq, test_size=0.3, random_state=42)

In [197]:
cat_model.fit(X_train, y_train)

voting_model = VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
voting_model.fit(X_train, y_train)

y_pred_cat = cat_model.predict(X_valid)
y_pred_voting = voting_model.predict(X_valid)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 98, number of used features: 5
[LightGBM] [Info] Start training from score 17.820918
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [198]:
result_df = pd.concat([X_valid, y_valid], axis=1)
result_df['CatBoost_Pred'] = y_pred_cat
result_df['Voting_Pred'] = y_pred_voting
result_df['Cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['voting-real'] = ( (result_df['가격(백만원)'] - result_df['Voting_Pred'])**2)**0.5

In [199]:
model_stats = result_df.groupby('모델')[['Cat-real', 'voting-real']].agg(['mean', 'std'])
model_stats

Cat-real           voting-real          
        mean       std        mean       std
모델                                          
4   4.700381  2.943622     4.66871  3.105838

In [200]:
len(result_df[result_df['배터리용량'].isna()]) , len(result_df)

(15, 42)

In [205]:
result_df['배터리용량'].max(), result_df['배터리용량'].min()

(88.08, 46.13)

In [207]:
result_df[result_df['배터리용량'].isna()]['Cat-real'].mean(),result_df[~result_df['배터리용량'].isna()]['Cat-real'].mean()

(5.404341785607309, 4.309290946274292)

In [201]:
result_df[result_df['Cat-real']>2].sort_values(by='Cat-real',ascending = False)

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),CatBoost_Pred,Voting_Pred,Cat-real,voting-real
51,2,4,2,66.27,1,59998,1,0,1,23.34,12.217271,13.033522,11.122729,10.306478
56,2,4,2,NaN,1,75303,1,0,2,11.27,22.174203,20.418535,10.904203,9.148535
45,2,4,1,65.21,1,44708,3,0,0,26.81,17.141006,18.138749,9.668994,8.671251
26,2,4,1,NaN,1,47892,3,0,0,9.92,18.919350,19.571998,8.999350,9.651998
131,2,4,2,NaN,1,35859,1,0,0,10.22,19.213237,18.449037,8.993237,8.229037
18,2,4,2,NaN,1,56809,1,0,1,27.42,19.904079,18.875983,7.515921,8.544017
110,2,4,1,57.83,1,72013,3,0,0,26.00,18.817824,18.555172,7.182176,7.444828
10,2,4,2,NaN,1,22289,1,0,0,9.77,16.935837,18.775722,7.165837,9.005722
127,2,4,2,NaN,1,21069,1,0,2,9.83,16.899058,18.811568,7.069058,8.981568
19,2,4,1,46.13,1,29087,3,0,0,15.00,21.851225,20.492173,6.851225,5.492173


In [208]:
train_tay = train[train['모델']==14].drop('ID',axis=1).reset_index(drop=True)

X_train_14 = train_tay.drop(columns=['가격(백만원)'])
y_train_14 = train_tay['가격(백만원)']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_14, y_train_14, test_size=0.2, random_state=42)

In [209]:
cat_model.fit(X_train, y_train)

voting_model = VotingRegressor([('cat', cat_model), ('lgb', lgb_model)])
voting_model.fit(X_train, y_train)

y_pred_cat = cat_model.predict(X_valid)
y_pred_voting = voting_model.predict(X_valid)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 117
[LightGBM] [Info] Number of data points in the train set: 288, number of used features: 5
[LightGBM] [Info] Start training from score 109.943090
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [210]:
result_df = pd.concat([X_valid, y_valid], axis=1)
result_df['CatBoost_Pred'] = y_pred_cat
result_df['Voting_Pred'] = y_pred_voting
result_df['Cat-real'] =( (result_df['가격(백만원)'] - result_df['CatBoost_Pred'])**2)**0.5
result_df['voting-real'] = ( (result_df['가격(백만원)'] - result_df['Voting_Pred'])**2)**0.5

In [181]:
model_stats = result_df.groupby('모델')[['Cat-real', 'voting-real']].agg(['mean', 'std'])
model_stats

Cat-real           voting-real          
        mean       std        mean       std
모델                                          
14  2.315062  2.221902    2.247834  2.187374

In [211]:
len(result_df[result_df['배터리용량'].isna()]) , len(result_df)

(29, 73)

In [212]:
result_df['배터리용량'].max()

90.0

In [213]:
result_df[result_df['배터리용량'].isna()]['Cat-real'].mean(), result_df[~result_df['배터리용량'].isna()]['Cat-real'].mean()

(3.963261823646175, 1.228747775532195)

In [185]:
result_df[result_df['Cat-real']>2].sort_values(by='Cat-real',ascending = False)

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),CatBoost_Pred,Voting_Pred,Cat-real,voting-real
124,4,14,0,90.000,0,7065,2,0,0,95.18,102.489684,102.952629,7.309684,7.772629
46,4,14,0,NaN,0,2329,2,1,0,95.56,102.351624,101.964471,6.791624,6.404471
22,4,14,0,NaN,0,7294,2,0,0,96.14,102.674108,102.785252,6.534108,6.645252
56,4,14,0,NaN,0,6049,2,0,0,104.50,98.018148,98.337052,6.481852,6.162948
175,4,14,0,90.000,0,4461,2,0,0,104.33,98.008504,98.751901,6.321496,5.578099
297,4,14,0,90.000,0,7532,2,0,0,95.90,102.170935,102.610261,6.270935,6.710261
232,4,14,0,NaN,0,2147,2,0,0,96.36,102.487342,102.032330,6.127342,5.672330
5,4,14,0,NaN,0,3660,2,0,0,104.57,98.498340,98.332994,6.071660,6.237006
224,4,14,0,NaN,0,5349,2,0,0,103.38,97.602310,97.459947,5.777690,5.920053
73,4,14,0,NaN,0,1367,2,0,0,96.46,102.073872,100.913916,5.613872,4.453916


In [186]:
result_df[result_df['Cat-real']<1].sort_values(by='Cat-real',ascending = False)

,제조사,모델,차량상태,배터리용량,구동방식,주행거리(km),보증기간(년),사고이력,연식(년),가격(백만원),CatBoost_Pred,Voting_Pred,Cat-real,voting-real
285,4,14,1,70.071,0,34139,1,0,1,120.01,120.988971,120.762982,0.978971,0.752982
303,4,14,1,67.268,0,21521,2,0,0,120.09,121.046654,120.772071,0.956654,0.682071
211,4,14,1,72.991,0,47283,0,0,2,121.69,120.763182,120.801046,0.926818,0.888954
298,4,14,1,64.578,0,40967,2,0,0,121.78,120.912322,121.004155,0.867678,0.775845
334,4,14,1,72.991,0,18701,0,0,2,121.43,120.610274,120.620969,0.819726,0.809031
341,4,14,2,59.515,0,189307,2,0,0,118.93,118.112917,118.218136,0.817083,0.711864
25,4,14,2,54.849,0,157354,2,0,0,113.95,114.646938,114.683627,0.696938,0.733627
173,4,14,2,54.849,0,169702,2,1,0,115.05,114.387602,114.603611,0.662398,0.446389
357,4,14,1,70.071,0,31595,1,0,1,120.48,121.118416,120.977019,0.638416,0.497019
218,4,14,1,64.578,0,37887,2,0,0,120.31,120.896268,120.963756,0.586268,0.653756
